<a href="https://colab.research.google.com/github/NilakshanRaveendran/bilingual-voice-translation-research/blob/main/notebooks/Dataset_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

BASE_DIR = "/content/drive/MyDrive/bilingual_voice_translation"
RAW_DIR = f"{BASE_DIR}/data/raw"
PROC_DIR = f"{BASE_DIR}/data/processed"
SEG_DIR = f"{BASE_DIR}/data/segments"

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PROC_DIR, exist_ok=True)
os.makedirs(SEG_DIR, exist_ok=True)

print("Folders created successfully")

Folders created successfully


In [3]:
!pip install librosa soundfile pydub noisereduce torchaudio wget datasets

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=3345e9967006bc17f442e67644ecd3f2a4620d008df9bfbe553deea4c4551998
  Stored in directory: /root/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [4]:
import os

RAW_DIR = "/content/drive/MyDrive/bilingual_voice_translation/data/raw"
os.makedirs(RAW_DIR, exist_ok=True)

print("Raw data directory ready")

Raw data directory ready


In [5]:
%cd /content

/content


In [6]:
# !wget https://storage.googleapis.com/cvss/cvss_c_v1.0/cvss_c_es_en_v1.0.tar.gz

In [7]:
# !ls -lh cvss_c_es_en_v1.0.tar.gz

In [8]:
# !tar -xvzf cvss_c_es_en_v1.0.tar.gz

In [9]:
# ls /content/train | head -n 100 | xargs -I {} mv /content/train/{} /content/drive/MyDrive/bilingual_voice_translation/data/raw/

In [10]:
import os

RAW_DIR = "/content/drive/MyDrive/bilingual_voice_translation/data/raw"
files = os.listdir(RAW_DIR)

print("Number of files in raw folder:", len(files))
print(files[:10])

Number of files in raw folder: 100
['common_voice_es_18307340.mp3.wav', 'common_voice_es_18306568.mp3.wav', 'common_voice_es_18306607.mp3.wav', 'common_voice_es_18306819.mp3.wav', 'common_voice_es_18307255.mp3.wav', 'common_voice_es_18306821.mp3.wav', 'common_voice_es_18308708.mp3.wav', 'common_voice_es_18307342.mp3.wav', 'common_voice_es_18307298.mp3.wav', 'common_voice_es_18307254.mp3.wav']


In [11]:
import os

RAW_DIR = "/content/drive/MyDrive/bilingual_voice_translation/data/raw"
STD_DIR = "/content/drive/MyDrive/bilingual_voice_translation/data/standardized"
CLEAN_DIR = "/content/drive/MyDrive/bilingual_voice_translation/data/cleaned"
SEG_DIR = "/content/drive/MyDrive/bilingual_voice_translation/data/segments"

os.makedirs(STD_DIR, exist_ok=True)
os.makedirs(CLEAN_DIR, exist_ok=True)
os.makedirs(SEG_DIR, exist_ok=True)

print("Folders ready")

Folders ready


In [12]:
import librosa
import soundfile as sf

TARGET_SR = 16000

for file in os.listdir(RAW_DIR):
    if file.endswith(".wav"):
        in_path = os.path.join(RAW_DIR, file)
        out_path = os.path.join(STD_DIR, file)

        audio, sr = librosa.load(in_path, sr=None, mono=True)

        if sr != TARGET_SR:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=TARGET_SR)

        sf.write(out_path, audio, TARGET_SR)

print("✅ Audio standardization completed (16 kHz, mono)")

✅ Audio standardization completed (16 kHz, mono)


In [13]:
import noisereduce as nr
import numpy as np

for file in os.listdir(STD_DIR):
    in_path = os.path.join(STD_DIR, file)
    out_path = os.path.join(CLEAN_DIR, file)

    audio, sr = librosa.load(in_path, sr=16000)

    reduced = nr.reduce_noise(y=audio, sr=sr)

    # normalize
    if np.max(np.abs(reduced)) > 0:
        reduced = reduced / np.max(np.abs(reduced))

    sf.write(out_path, reduced, sr)

print("✅ Noise reduction & normalization completed")

✅ Noise reduction & normalization completed


In [14]:
from pydub import AudioSegment
from pydub.silence import split_on_silence

def segment_audio(file_path, out_dir):
    audio = AudioSegment.from_wav(file_path)

    chunks = split_on_silence(
        audio,
        min_silence_len=500,     # 0.5 sec silence
        silence_thresh=-40,      # silence threshold (dB)
        keep_silence=200         # keep short pause
    )

    base = os.path.splitext(os.path.basename(file_path))[0]

    for i, chunk in enumerate(chunks):
        chunk.export(
            os.path.join(out_dir, f"{base}_seg_{i}.wav"),
            format="wav"
        )

for file in os.listdir(CLEAN_DIR):
    segment_audio(os.path.join(CLEAN_DIR, file), SEG_DIR)

print("✅ Silence-based segmentation completed")

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


✅ Silence-based segmentation completed


In [15]:
segments = os.listdir(SEG_DIR)

print("Total segments created:", len(segments))
print("Sample segment files:", segments[:10])

Total segments created: 103
Sample segment files: ['common_voice_es_18306544.mp3_seg_0.wav', 'common_voice_es_18306545.mp3_seg_0.wav', 'common_voice_es_18306546.mp3_seg_0.wav', 'common_voice_es_18306547.mp3_seg_0.wav', 'common_voice_es_18306548.mp3_seg_0.wav', 'common_voice_es_18306548.mp3_seg_1.wav', 'common_voice_es_18306565.mp3_seg_0.wav', 'common_voice_es_18306566.mp3_seg_0.wav', 'common_voice_es_18306567.mp3_seg_0.wav', 'common_voice_es_18306568.mp3_seg_0.wav']


In [16]:
import pandas as pd

rows = []

for f in segments:
    path = os.path.join(SEG_DIR, f)
    audio, sr = librosa.load(path, sr=16000)
    duration = librosa.get_duration(y=audio, sr=sr)
    rows.append([f, duration])

df = pd.DataFrame(rows, columns=["Segment", "Duration (seconds)"])
df.head()

,Segment,Duration (seconds)
0,common_voice_es_18306544.mp3_seg_0.wav,1.438
1,common_voice_es_18306545.mp3_seg_0.wav,3.112
2,common_voice_es_18306546.mp3_seg_0.wav,1.112
3,common_voice_es_18306547.mp3_seg_0.wav,2.462
4,common_voice_es_18306548.mp3_seg_0.wav,3.243
